In [116]:
import sklearn
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [8]:
df = pd.read_csv('../input/train_predictions.csv')
df.head(3)

,StudyInstanceUID,Slice,ImageHeight,ImageWidth,SliceThickness,ImagePositionPatient_x,ImagePositionPatient_y,ImagePositionPatient_z,C1,C2,...,C5_effnet_frac,C6_effnet_frac,C7_effnet_frac,C1_effnet_vert,C2_effnet_vert,C3_effnet_vert,C4_effnet_vert,C5_effnet_vert,C6_effnet_vert,C7_effnet_vert
0,1.2.826.0.1.3680043.10032,1,512,512,0.625,-82.0,11.712,37.166,0,0,...,0.000693,0.000422,0.000429,0.001040,0.001831,0.003172,0.007347,0.001095,0.000229,0.000194
1,1.2.826.0.1.3680043.10032,2,512,512,0.625,-82.0,11.712,36.541,0,0,...,0.000600,0.000361,0.000354,0.000922,0.001411,0.002350,0.005200,0.000752,0.000163,0.000147
2,1.2.826.0.1.3680043.10032,3,512,512,0.625,-82.0,11.712,35.916,0,0,...,0.000624,0.000379,0.000424,0.000915,0.001315,0.002192,0.004757,0.000789,0.000190,0.000194


In [7]:
df_y = pd.read_csv(f'../input/rsna-2022-cervical-spine-fracture-detection/train.csv')
df_y = df_y.query('StudyInstanceUID != "1.2.826.0.1.3680043.20574"').reset_index(drop=True)
df_y.head(3)

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0


In [111]:
c1c7 = [f'C{i}' for i in range(1, 8)]

In [221]:
df_y = df_y[:1400] # 取1k4之后的做测试
df_y

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1395,1.2.826.0.1.3680043.611,1,0,0,1,0,0,0,0
1396,1.2.826.0.1.3680043.7247,1,0,0,0,0,0,0,1
1397,1.2.826.0.1.3680043.15891,1,0,0,0,0,0,1,0
1398,1.2.826.0.1.3680043.10041,1,0,0,0,0,1,1,1


In [164]:
# 输入 7x1  输出 1x1
# 先拿所有C1来看 1.拿出所有C1的slice 2.把这些slice的frac值平均  -> 7x1    target是这个样本的C1
UID_target_dict = {}
for i in range(7):
    for row in df_y.iterrows():
        UID_target_dict[row[1][0]+'_'+c1c7[i]] = row[1][i+2]
len(UID_target_dict)

9800

In [165]:
UID_target_dict

{'1.2.826.0.1.3680043.6200_C1': 1,
 '1.2.826.0.1.3680043.27262_C1': 0,
 '1.2.826.0.1.3680043.21561_C1': 0,
 '1.2.826.0.1.3680043.12351_C1': 0,
 '1.2.826.0.1.3680043.1363_C1': 0,
 '1.2.826.0.1.3680043.4859_C1': 0,
 '1.2.826.0.1.3680043.18659_C1': 0,
 '1.2.826.0.1.3680043.17364_C1': 0,
 '1.2.826.0.1.3680043.849_C1': 0,
 '1.2.826.0.1.3680043.24045_C1': 0,
 '1.2.826.0.1.3680043.4744_C1': 0,
 '1.2.826.0.1.3680043.15773_C1': 1,
 '1.2.826.0.1.3680043.24946_C1': 0,
 '1.2.826.0.1.3680043.9290_C1': 0,
 '1.2.826.0.1.3680043.5482_C1': 1,
 '1.2.826.0.1.3680043.11090_C1': 0,
 '1.2.826.0.1.3680043.13810_C1': 0,
 '1.2.826.0.1.3680043.28252_C1': 0,
 '1.2.826.0.1.3680043.26052_C1': 0,
 '1.2.826.0.1.3680043.4131_C1': 0,
 '1.2.826.0.1.3680043.9996_C1': 0,
 '1.2.826.0.1.3680043.13814_C1': 0,
 '1.2.826.0.1.3680043.9121_C1': 0,
 '1.2.826.0.1.3680043.9447_C1': 0,
 '1.2.826.0.1.3680043.7824_C1': 0,
 '1.2.826.0.1.3680043.4561_C1': 0,
 '1.2.826.0.1.3680043.28657_C1': 0,
 '1.2.826.0.1.3680043.22438_C1': 0,
 '1.2.

In [166]:
UID_x_dict = {}
vert_c1c7 = [f'C{i}_effnet_vert' for i in range(1, 8)]
frac_c1c7 = [f'C{i}_effnet_frac' for i in range(1, 8)]
for UID_C in tqdm(UID_target_dict.keys()):
    UID = UID_C.split('_C')[0]
    cn = int(UID_C.split('_C')[1])
    df1 = df.loc[df['StudyInstanceUID']==UID].copy()
    df1[vert_c1c7] = (df1[vert_c1c7] > 0.5).astype(int)

    x = np.average(df1.loc[df1[f'C{cn}_effnet_vert']>0.5][frac_c1c7].values,axis=0).tolist()
    UID_x_dict[UID_C] =  x
len(UID_x_dict)

  0%|          | 0/9800 [00:00<?, ?it/s]

W:\Anaconda3\envs\rsna39\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
W:\Anaconda3\envs\rsna39\lib\site-packages\numpy\core\_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
W:\Anaconda3\envs\rsna39\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
W:\Anaconda3\envs\rsna39\lib\site-packages\numpy\core\_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
W:\Anaconda3\envs\rsna39\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
W:\Anaconda3\envs\rsna39\lib\site-packages\numpy\core\_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
W:\Anaconda3\envs\rsna39\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  a

9800

In [167]:
y = np.array([[]]).T
x = np.array([[]]*7).T
for UID in UID_target_dict.keys():
    x = np.vstack((x,np.array(UID_x_dict[UID]).T))
    y = np.vstack((y,np.array(UID_target_dict[UID]).T))
y = y.ravel()
x.shape

(9800, 7)

In [168]:
x = np.nan_to_num(x)
x = np.maximum(x, 0.000001)
x = np.minimum(x,0.999999)

In [316]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.svm import SVC

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3,random_state=42)
len(ytrain)

6860

In [177]:
rf=RandomForestClassifier(n_estimators=1000)
# rf = rf.fit(xtrain,ytrain)
rf = rf.fit(x,y)

score_c = rf.score(xtest, ytest)

In [184]:
ada = AdaBoostClassifier()
ada = ada.fit(xtrain,ytrain)

score_c = ada.score(xtest, ytest)

In [320]:
gb = GradientBoostingClassifier(n_estimators=188)
gb = gb.fit(x,y)
gb.score(xtest, ytest)

0.9200680272108843

In [319]:
svm_model = SVC(kernel = 'linear',probability=True)
svm_model.fit(xtrain,ytrain)

score_c = svm_model.score(xtest, ytest)

In [280]:
score_c

0.5523809523809524

In [321]:
y_test_proba_rfc = gb.predict_proba(xtest)
false_positive_rate_rfc, recall_rfc, thresholds_rfc = roc_curve(ytest, y_test_proba_rfc[:, 1])
auc(false_positive_rate_rfc, recall_rfc)

0.8629233194995298

In [322]:
import joblib
#lr是一个LogisticRegression模型
joblib.dump(gb, 'gb.model')
# lr = joblib.load('lr.model')

['gb.model']

C1-C7 预测patient overall

In [283]:
rf_pred = gb.predict_proba(x)


In [284]:
UID_ypred_dict = {}
i=0
for UID_C in tqdm(UID_target_dict.keys()):
    UID = UID_C.split('_C')[0]
    cn = int(UID_C.split('_C')[1])
    if UID in UID_ypred_dict.keys():
        UID_ypred_dict[UID][cn-1] = rf_pred[i,1]
    else:
        UID_ypred_dict[UID] = [0.,0.,0.,0.,0.,0.,0.]
        UID_ypred_dict[UID][cn-1] = rf_pred[i,1]
    i+=1
UID_ypred_dict.__len__()

  0%|          | 0/9800 [00:00<?, ?it/s]

1400

In [285]:
UID_overall_dict = {}
for row in df_y.iterrows():
    UID_overall_dict[row[1][0]] = row[1][1]


In [286]:
y_overall = np.array([[]]).T
x_c1c7 = np.array([[]]*7).T
for UID in UID_ypred_dict.keys():
    x_c1c7 = np.vstack((x_c1c7,np.array(UID_ypred_dict[UID]).T))
    y_overall = np.vstack((y_overall,np.array(UID_overall_dict[UID]).T))
y_overall = y_overall.ravel()
x_c1c7.shape

(1400, 7)

In [294]:
xtrain, xtest, ytrain, ytest = train_test_split(x_c1c7, y_overall, test_size=0.3,random_state=42)
len(ytrain)

980

In [288]:
gb = GradientBoostingClassifier(n_estimators=121)
gb = gb.fit(x,y)
gb.score(xtest, ytest)

0.5761904761904761

In [313]:
rf=RandomForestClassifier(n_estimators=333,max_depth=4)
# rf = rf.fit(xtrain,ytrain)
rf = rf.fit(xtrain,ytrain)
rf.score(xtest, ytest)

0.7857142857142857

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
gpc = GaussianProcessClassifier(1.0 * RBF(1.0))
gpc = gpc.fit(xtrain,ytrain)
gpc.score(xtest, ytest)

In [314]:
y_test_proba_rfc = rf.predict_proba(xtest)
false_positive_rate_rfc, recall_rfc, thresholds_rfc = roc_curve(ytest, y_test_proba_rfc[:, 1])
auc(false_positive_rate_rfc, recall_rfc)

0.8481905842314237

In [315]:
joblib.dump(rf, 'rf2.model')

['rf2.model']